# Using DebugResult

Here, we will show how to use `DebugResult` to debug some problems we might encounter when using our mlir-opt CLI Wrapper.

Let’s first import some necessary classes and generate an instance of our mlir-opt CLI Wrapper.

In [1]:
from mlir_graphblas import MlirOptCli

cli = MlirOptCli(executable=None, options=None)

## Generate Example Input

Let's say we have a bunch of MLIR code that we're not familiar with. 

In [2]:
mlir_string = """
#trait_sum_reduction = {
  indexing_maps = [
    affine_map<(i,j,k) -> (i,j,k)>,  // A
    affine_map<(i,j,k) -> ()>        // x (scalar out)
  ],
  iterator_types = ["reduction", "reduction", "reduction"],
  doc = "x += SUM_ijk A(i,j,k)"
}

#sparseTensor = #sparse_tensor.encoding<{
  dimLevelType = [ "compressed", "compressed", "compressed" ],
  dimOrdering = affine_map<(i,j,k) -> (i,j,k)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @func_f32(%argA: tensor<10x20x30xf32, #sparseTensor>) -> f32 {
  %out_tensor = constant dense<0.0> : tensor<f32>
  %reduction = linalg.generic #trait_sum_reduction
     ins(%argA: tensor<10x20x30xf32, #sparseTensor>)
    outs(%out_tensor: tensor<f32>) {
      ^bb(%a: f32, %x: f32):
        %0 = addf %x, %a : f32
        linalg.yield %0 : f32
  } -> tensor<f32>
  %answer = tensor.extract %reduction[] : tensor<f32>
  return %answer : f32
}
"""
mlir_bytes = mlir_string.encode()

Since we're not familiar with this code, we don't exactly know what passes are necessary or in what order they should go in.

Let's say that this is the first set of passes we try. 

In [3]:
passes = [
    "--sparsification",
    "--sparse-tensor-conversion",
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-memref-to-llvm",
    "--convert-std-to-llvm",
]

Let's see what results we get. 

In [4]:
result = cli.apply_passes(mlir_bytes, passes)

MlirOptError: <stdin>:20:16: error: failed to legalize operation 'builtin.unrealized_conversion_cast' that was explicitly marked illegal
  %reduction = linalg.generic #trait_sum_reduction
               ^

We get an exception. 

Unfortunately, the exception message isn't very clear as it only gives us the immediate error message but doesn't inform us of the context in which it occurred, e.g. in which pass the error occurred (if any) or if any necessary passes are missing. 

We only know that the operation `builtin.unrealized_conversion_cast` shows up somewhere and that it's a problem.

Let's try to use the `debug_passes` method instead of the `apply_passes` to get more information. 

In [5]:
result = cli.debug_passes(mlir_bytes, passes)

In [6]:
result

  Error when running convert-std-to-llvm  
<stdin>:77:13: error: failed to legalize operation 'builtin.unrealized_conversion_cast' that was explicitly marked illegal
      %62 = builtin.unrealized_conversion_cast %arg1 : index to i64
            ^
<stdin>:77:13: note: see current operation: %72 = "builtin.unrealized_conversion_cast"(%arg1) : (index) -> i64 loc("<stdin>":77:13)


  Input to convert-std-to-llvm  
             10        20        30        40        50        60        70        80        90        100       110       120       130       140       
    1234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345
    -------------------------------------------------------------------------------------------------------------------------------------------------
  1|builtin.module  {
  2|  llvm.func @memrefCopy(i64, !llvm.ptr<struct<(i64, ptr<i8>)>>, !llvm.ptr<struct<(i64, ptr<i8>)>>)
  3|  

This large output may seem intimidating due to it's size, but it's mostly just large since it's showing the inputs to each pass. 

We know that the error happens when the `builtin.unrealized_conversion_cast` operation occurs. 

We can see from the output above that it happens during the `convert-std-to-llvm` pass. 

It's likely that there's something problematic in the input to that pass, so it's worth looking into the IR that was given to the `convert-std-to-llvm` pass, which we can see under the section labelled `Input to convert-std-to-llvm`. We'll show a sort snippet of it below. 

In [7]:
result_string = str(result)
print("\n".join(result_string.splitlines()[9:30]))

  Input to convert-std-to-llvm  
             10        20        30        40        50        60        70        80        90        100       110       120       130       140       
    1234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345
    -------------------------------------------------------------------------------------------------------------------------------------------------
  1|builtin.module  {
  2|  llvm.func @memrefCopy(i64, !llvm.ptr<struct<(i64, ptr<i8>)>>, !llvm.ptr<struct<(i64, ptr<i8>)>>)
  3|  llvm.func @malloc(i64) -> !llvm.ptr<i8>
  4|  llvm.mlir.global private constant @__constant_xf32(0.000000e+00 : f32) : f32
  5|  builtin.func private @sparseValuesF32(!llvm.ptr<i8>) -> memref<?xf32>
  6|  builtin.func private @sparsePointers64(!llvm.ptr<i8>, index) -> memref<?xi64>
  7|  builtin.func @func_f32(%arg0: !llvm.ptr<i8>) -> f32 {
  8|    %c2 = constant 2 : index
  9|  

While this is a good idea in general, it doesn't seem to be useful here. When MLIR applies a pass, that pass is applied until quiescence, i.e. it keeps applying the pass until nothing changes (or until some limit on the number of applications is reached). 

It seems that the `convert-std-to-llvm` pass has already been applied a few times since we see several ops from the LLVM dialect already present in the IR shown under the `Input to convert-std-to-llvm` section (for example, we see `llvm.mlir.constant`). 

Another good place to look is in the output of the last pass right before we get our error. Let's look at the result of the `convert-memref-to-llvm` pass. 

In [8]:
print("\n".join(result_string.splitlines()[139:187]))

  Input to convert-memref-to-llvm  
builtin.module  {
  memref.global "private" constant @__constant_xf32 : memref<f32> = dense<0.000000e+00>
  builtin.func private @sparseValuesF32(!llvm.ptr<i8>) -> memref<?xf32>
  builtin.func private @sparsePointers64(!llvm.ptr<i8>, index) -> memref<?xi64>
  builtin.func @func_f32(%arg0: !llvm.ptr<i8>) -> f32 {
    %c2 = constant 2 : index
    %c0 = constant 0 : index
    %c1 = constant 1 : index
    %0 = memref.get_global @__constant_xf32 : memref<f32>
    %1 = call @sparsePointers64(%arg0, %c0) : (!llvm.ptr<i8>, index) -> memref<?xi64>
    %2 = call @sparsePointers64(%arg0, %c1) : (!llvm.ptr<i8>, index) -> memref<?xi64>
    %3 = call @sparsePointers64(%arg0, %c2) : (!llvm.ptr<i8>, index) -> memref<?xi64>
    %4 = call @sparseValuesF32(%arg0) : (!llvm.ptr<i8>) -> memref<?xf32>
    %5 = memref.alloc() : memref<f32>
    memref.copy %0, %5 : memref<f32> to memref<f32>
    %6 = memref.load %1[%c0] : memref<?xi64>
    %7 = index_cast %6 : i64 to index
 

We see that the ops are mostly ops from te standard dialect. However, there are some ops from the `scf` dialect. It's unclear whether or not the `convert-std-to-llvm` dialect can handle ops from the `scf` dialect. Given the name of the `convert-std-to-llvm` pass, we can infer that it can only handle ops from the `std` dialect and cannot handle ops from the `scf` dialect. Let's see if there are any passes that can convert from the `scg` dialect to the `std` dialect. 

In [9]:
!mlir-opt --help | grep "scf"

Available Dialects: acc, affine, amx, arm_neon, arm_sve, async, builtin, complex, dlti, emitc, gpu, linalg, llvm, math, memref, nvvm, omp, pdl, pdl_interp, quant, rocdl, scf, shape, sparse_tensor, spv, std, tensor, test, tosa, vector, x86vector
      --async-parallel-for                              -   Convert scf.parallel operations to multiple async compute ops executed concurrently for non-overlapping iteration ranges
      --convert-linalg-tiled-loops-to-scf               -   Lower linalg tiled loops to SCF loops and parallel loops
      --convert-openacc-to-scf                          -   Convert the OpenACC ops to OpenACC with SCF dialect
      --convert-parallel-loops-to-gpu                   -   Convert mapped scf.parallel ops to gpu launch operations
      --convert-scf-to-openmp                           -   Convert SCF parallel loop to OpenMP parallel + workshare constructs.
      --convert-scf-to-spirv                            -   Convert SCF dialect to SPIR-V dialect.


The pass `convert-scf-to-std` seems promising as it intends to convert the `scf` dialect to `std` dialect. 

Let's see if running the `convert-scf-to-std` pass right before the `convert-std-to-llvm` pass will get rid of our exception. 

In [10]:
passes = [
    "--sparsification",
    "--sparse-tensor-conversion",
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-memref-to-llvm",
    "--convert-scf-to-std", # newly added
    "--convert-std-to-llvm",
]
result = cli.apply_passes(mlir_bytes, passes)
print(result[:1500])

builtin.module attributes {llvm.data_layout = ""}  {
  llvm.func @memrefCopy(i64, !llvm.ptr<struct<(i64, ptr<i8>)>>, !llvm.ptr<struct<(i64, ptr<i8>)>>)
  llvm.func @malloc(i64) -> !llvm.ptr<i8>
  llvm.mlir.global private constant @__constant_xf32(0.000000e+00 : f32) : f32
  llvm.func @sparseValuesF32(!llvm.ptr<i8>) -> !llvm.struct<(ptr<f32>, ptr<f32>, i64, array<1 x i64>, array<1 x i64>)> attributes {sym_visibility = "private"}
  llvm.func @sparsePointers64(!llvm.ptr<i8>, i64) -> !llvm.struct<(ptr<i64>, ptr<i64>, i64, array<1 x i64>, array<1 x i64>)> attributes {sym_visibility = "private"}
  llvm.func @func_f32(%arg0: !llvm.ptr<i8>) -> f32 {
    %0 = llvm.mlir.constant(2 : index) : i64
    %1 = llvm.mlir.constant(0 : index) : i64
    %2 = llvm.mlir.constant(1 : index) : i64
    %3 = llvm.mlir.constant(1 : index) : i64
    %4 = llvm.mlir.null : !llvm.ptr<f32>
    %5 = llvm.getelementptr %4[%3] : (!llvm.ptr<f32>, i64) -> !llvm.ptr<f32>
    %6 = llvm.ptrtoint %5 : !llvm.ptr<f32> to i64
  

It looks like it fixed our issue!